Pipeline for Extracting Keywords from the dataset

In [4]:
import pandas as pd
import spacy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np

In [6]:
nlp = spacy.load("en_core_web_sm")

path = "/Users/ethanvirtudazo/Desktop/DS105_Dataset/6622_jobs.xls"
df_import = pd.read_excel(path)

df_import.shape


(6623, 11)

In [8]:

df1 = df_import.iloc[:, 1:5]
df1.head()

df2 = df_import.iloc[:, [7,9,10]]
df2.head()

pdList = [df1,df2]
df = pd.concat(pdList,axis=1)
df.head()

df.shape

types = df.dtypes
print(types)

df_text = df.iloc[:, [0,1,6]]
df_text.head()

title                  object
details                object
deadline       datetime64[ns]
opport_type            object
location               object
company                object
links                  object
dtype: object


,title,details,links
0,Rothschild & Co - Private Equity Long-Term Int...,This London-based 6-month internship is an exc...,https://careers.lse.ac.uk//students/jobs/detai...
1,2023 HSBC Global Graduate Programme (Hong Kong...,You’re excited about starting your career and ...,https://careers.lse.ac.uk//students/jobs/detai...
2,2023 HSBC Global Internship Programme (Hong Ko...,You’re excited about starting your career and ...,https://careers.lse.ac.uk//students/jobs/detai...
3,"Graduate Training Scheme, Capital Markets",Graduate Training Scheme – LondonGreySpark Par...,https://careers.lse.ac.uk//students/jobs/detai...
4,6-Months Internship – Sell-side Tech M&A,"At IPTP, we understand software from decades o...",https://careers.lse.ac.uk//students/jobs/detai...
